# Analyizing Malignant / Benign tissues

Purpose of Level 4 Analysis_May26:
* Analysis includes:
    * Segmentation with MATLAB script run 

Purpose of Level 3 Analysis:
* Analysis includes:
    * Includes TZ and PZ analysis
    * Analyzes based off GS
    * normalizing, standardizing, both (1: normalize, 2: standardize)
    * removes when pixels, say LWF = 0
    * Weights images with multiplication factor
    * Weights HR images based on 

## Imports & User File Locations Settings

In [ ]:
''' Imports for the function to run'''

# %matplotlib widget
%matplotlib inline 
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import glob
import time
from scipy.io import loadmat
from skimage.transform import resize as ski_resize
import cv2
import copy
import seaborn as sns # for plotting w/ multiple y-axis (just two)
import math
import imageio
import matplotlib.patches as mpatches
import matlab.engine 

'''Selection of user definted inputs
    Including: 
        - folder inputs for ROIs
        - folder containing the analyzed data i.e. LWI maps '''

# Possible ROI folders to choose from (can be .mat or .png)
# -- ROI folders (drawn)
folderFullRois =     "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/2 ROIs/CombinedROIs/Full"
folderPZRois =       "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/2 ROIs/CombinedROIs/PZ"
folderPZMaligRois =  "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/2 ROIs/CombinedROIs/PZM"
# -- ROI folders (predicted & subset of patients)
folderPZml =         "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/7 Model Outputs/20210726_2231_UNETv2_cuda_PZ_test/Predicted_k3"
folderFullml =       "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/7 Model Outputs/20210718_1813_UNET_cpu_full_test/Predicted_k3"
# -- Gleason Score map
folder_gs =           "C:/Users/candi/Documents/Research/1 LWI Project/1 Data/2 ROIs/CombinedROIs/GSMap"

# Selecting the folder for ROI
folder_roi = folderFullRois
folder_pz = folderPZRois
# folder_roi = folderFullml
# folder_pz = folderPZml


# -- LWI maps 
folder_nnls_matlab = 'C:/Users/candi/Documents/Research/1 LWI Project/1 Data/8 NNLS Output/NNLS_matlab/LWI_nomask_Matlab(1)'
folder_nnls_decaes = 'C:/Users/candi/Documents/Research/1 LWI Project/1 Data/8 NNLS Output/DECAES_qt2'
#'C:/Users/candi/Documents/Research/1 LWI Project/1 Data/8 NNLS Output/DECAES_matlab/LWI_matlab'

# -- T2 Decay data
T2Decay_data_folder = 'C:/Users/candi/Documents/Research/1 LWI Project/1 Data/3 T2 Decay'

# -- High-Res T2w (512 x 512) Image data
HR_T2W_data_folder = 'C:/Users/candi/Documents/Research/1 LWI Project/1 Data/4 HighRes/High Res Mat'

# -- Location to save information in
folderOutputIms =    "C:/Users/candi/Documents/Research/1 LWI Project/3 Python Scripts/JupyterLab/Analysis/Analysis_level3"

# --- Setting colormap of images to always be gray
plt.rcParams['image.cmap'] = 'gray'

## Functions required

In [ ]:
# --------------------------------        
# ----- load_P_S
# --------------------------------
def load_P_S(lwiPath, folder_roi, folder_pz, folder_gs, pnum, snum):
    """ A function to load ROI and LWI data based on patient number and string
        Requires that the structure of the files are "P(#patient number)_S(#slice number).mat"
        Inputs:
            - lwiPath:      path to the LWI file folders (str)
            - folder_roi:   path to the full ROI file folders (str) 
            - folder_pz:    path to the PZ ROI files (str)
            - folder_gs:    path to the ROI files that malignancy levels based on GS (str)
            - pnum:         patient number (int)
            - snum:         slice number (int)
        Returns:
            - LWI:        the file with LWI information
            - fields:     string set with object names the LWIL file contains
            - roi:        the ROI corresponding to the LWI dataset returned
            - roiPZ:      the ROI corresponding to peripheral zone
            - roiTZ:      the ROI corresponding to transition zone
            - roiGS:      the malignant ROI corresponding to the LWI dataset returned"""    
    fields,lwi = loadDecaesOrMatlab(lwiPath, pnum, snum) # obtaining the fields and LWI dataset (decaes or NNLS)
#     roi, roiM = loadROI(roiPath, roiMaligPath, pnum, snum)
    roi_all, roi_pz, roi_tz, gs_map = loadROIall(folder_roi,folder_pz,folder_gs,pnum,snum)
#     # filtering out LWF = 0 through redrawing roi ********************************************************************************************************************* temp!
#     roi[np.where(np.isclose(lwi['LWF'], 0))] = False
#     roiM[np.where(np.isclose(lwi['LWF'], 0))] = False
    
    return fields, lwi, roi_all, roi_pz, roi_tz, gs_map

# --------------------------------        
# ----- loadROI
# --------------------------------
def loadROIall(folder_roi, folder_pz, folder_gs, pnum, snum):
    """A function to load the ROIs of the file given the patient number and slice number
       Inputs:
           - folder_roi:   path to the full ROI file folders (str) 
           - folder_pz:    path to the PZ ROI files (str)
           - folder_gs:    path to the ROI files that malignancy levels based on GS (str)
           - pnum:   patient number (int)
           - snum:   slice number (int)
       Returns:
           - roi:    ROI for the entire zone [bool]
           - roi_pz: the ROI for peripheral zone [bool]
           - roi_tz: the ROI for transition zone [bool]
           - gs_map: a map outline gleason grading in the slice"""
    # Determining the type of ROI's whether they are .mat or in .png format and loading. (All ROI)
    try:
        if glob.glob(folder_roi + '/P*' + str(pnum) + '_S*' + str(snum) + '*')[0].find('.mat') >0:
            roi = loadmat(glob.glob(folder_roi + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')[0])
            roi = roi[list(roi.keys())[3]]
        elif glob.glob(folder_roi + '/P*' + str(pnum) + '_S*'+ str(snum) + '*')[0].find('.png') >0:
            roi = cv2.imread(glob.glob(folder_roi + '/P*' + str(pnum) + '_S*' + str(snum) + '*.png')[0])
            roi = roi[:,:,0]
        else:
            raise ValueError('Error:  Problem reading ROI folder: ' + glob.glob(path + '/P*' + str(p) + '_S*')[0])
    except:
        print('Error:  All ROI file (P' + str(pnum) + '_S' + str(snum) + 'could not be found: ' + folder_roi + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')
        raise


    # Determining the type of ROI's whether they are .mat or in .png format and loading. (PZ ROI)
    try:
        if glob.glob(folder_pz + '/P*' + str(pnum) + '_S*' + str(snum) + '*')[0].find('.mat') >0:
            roi_pz = loadmat(glob.glob(folder_pz + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')[0])
            roi_pz = roi_pz[list(roi_pz.keys())[3]]
        elif glob.glob(folder_pz + '/P*' + str(pnum) + '_S*'+ str(snum) + '*')[0].find('.png') >0:
            roi_pz = cv2.imread(glob.glob(folder_pz + '/P*' + str(pnum) + '_S*' + str(snum) + '*.png')[0])
            roi_pz = roi_pz[:,:,0]
        else:
            raise ValueError('Error:  Problem reading ROI folder: ' + glob.glob(path + '/P*' + str(p) + '_S*')[0])
    except:
        print('Error:  PZ ROI file (P' + str(pnum) + '_S' + str(snum) + 'could not be found: ' + folder_pz + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')
        raise

    # Loading GS map
    try:
        if glob.glob(folder_gs + '/P*' + str(pnum) + '_S*' + str(snum) + '*')[0].find('.mat') >0:
            gs_map = loadmat(glob.glob(folder_gs + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')[0])
            gs_map = gs_map[list(gs_map.keys())[3]]
        else:
            raise ValueError('Error:  Problem reading ROI folder: ' + glob.glob(path + '/P*' + str(p) + '_S*')[0])
    except:
        print('Error:  GS Map file (P' + str(pnum) + '_S' + str(snum) + 'could not be found: ' + folder_gs + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')
        raise

    if roi_pz is not None:
        roi_tz = cleanROI(roi.astype(bool) & ~roi_pz.astype(bool))
        
    return roi.astype(bool), roi_pz.astype(bool), roi_tz.astype(bool), gs_map

# --------------------------------        
# ----- loadROI
# --------------------------------
def loadROI(path, pathM, pnum, snum):
    """A function to load the ROIs of the file given the patient number and slice number
       Inputs:
           - path:   a path for where the ROI files are located (str)
           - pathM:  a path for the malignant ROI files location (str_)
           - pnum:   patient number (int)
           - snum:   slice number (int)
       Returns:
           - roi:  ROI for the entire zone [bool]
           - roiM: the ROI for the malignant pixels given a patient + slice  [bool]"""
    # Determining the type of ROI's whether they are .mat or in .png format
    try:
        if glob.glob(path + '/P*' + str(pnum) + '_S*' + str(snum) + '*')[0].find('.mat') >0:
            roi = loadmat(glob.glob(path + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')[0])
            roi = roi[list(roi.keys())[3]]
        elif glob.glob(path + '/P*' + str(pnum) + '_S*'+ str(snum) + '*')[0].find('.png') >0:
            roi = cv2.imread(glob.glob(folder_roi + '/P*' + str(pnum) + '_S*' + str(snum) + '*.png')[0])
            roi = roi[:,:,0]
        else:
            raise ValueError('Error:  Problem reading ROI folder: ' + glob.glob(path + '/P*' + str(p) + '_S*')[0])
    except:
        print('Error:  ROI file (P' + str(pnum) + '_S' + str(snum) + 'could not be found: ' + path + '/P*' + str(pnum) + '_S*' + str(snum) + '*.mat')
        raise
    
    # Determining the type of malignant ROI's whether they are .mat or in .png format
    # Does not raise an exception because not all rois contain malignancy! If so, the mask will be all false
    try: 
        if glob.glob(pathM + '/P*' + str(pnum) + '_S' + str(snum) + '*')[0].find('.mat') >0:
            roiM = loadmat(glob.glob(pathM + '/P*' + str(pnum) + '_S' + str(snum) + '*.mat')[0])
            roiM = roiM[list(roiM.keys())[3]]
        elif glob.glob(pathM + '/P*' + str(pnum) + '_S' + str(snum) + '*')[0].find('.png') >0:
            roiM = cv2.imread(glob.glob(pathM + '/P*' + str(pnum) + '_S' + str(snum) + '*.png')[0])
            roiM = roiM[:,:,0]
    except:
        roiM = np.zeros(np.shape(roi))
    
    return roi.astype(bool), roiM.astype(bool)



# --------------------------------        
# ----- loadDecaesOrMatlab
# --------------------------------
def loadDecaesOrMatlab(path, pnum, snum):
    """A function to determine whether the path is directed to the DECAES LWI data or NNLS and return
       the objects and the file itself.
       Inputs:
           - path:   a path for where the files are located (str)
           - pnum:       patient number (int)
           - snum:       slice number (int)
       Returns:
           - fields: string set with object names the LWIL file contains depending on NNLS or DECAES analysis
           - lwi:    the LWI file with the maps""" 
    lwipath = path + '/P*' +  str(pnum) + '*.mat'
    try:
        if 'results' in h5py.File(glob.glob(lwipath)[0]):
            if len(glob.glob(path + '/P*' +  str(pnum) + '_S*' + str(snum) + '.mat')) > 0: # loading matlab data
                lwi = reorder_h5py(h5py.File(glob.glob(path + '/P*' +  str(pnum) + '_S*' + str(snum) + '.mat')[0])['results'],snum)
            elif len(glob.glob(path + '/P*' +  str(pnum) + '*.mat')) > 0: # loading decaes, 7 param (nnls) data
                lwi = reorder_h5py(h5py.File(glob.glob(path + '/P*' + str(pnum) + '*.mat')[0])['results'],snum)
                
        else:
            lwi = reorder_h5py(h5py.File(glob.glob(path + '/P*' + str(pnum) + '*t2parts.mat')[0]),snum)
            lwi = {**lwi,**(reorder_h5py(h5py.File(glob.glob(path + '/P*' + str(pnum) + '*t2maps.mat')[0]),snum))} # adding two dictionaries together
    except: 
        print('Error:  No LWI map found for P' + str(pnum) + '_S' + str(snum) + ' given path: ' + path)
        raise
    fields = list(lwi.keys())
    return fields, lwi



# --------------------------------        
# ----- cleanROI
# --------------------------------
def cleanROI(roi):
    """ Removes small islands outside of ROI
    Input:
        - roi: the binary, boolean ROI that will be edited
    Returns:
        - cleaned_roi: the binary, boolean ROI that with small islands removed"""
    contours,_ = cv2.findContours((np.uint8(roi)), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cmax = max(contours, key = cv2.contourArea) 
    epsilon = 0.002 * cv2.arcLength(cmax, True)
    approx = cv2.approxPolyDP(cmax, epsilon, True)

    width, height = (np.uint8(roi)).shape

    #fill maximum contour and draw   
    img = np.zeros( [width, height, 3],dtype=np.uint8 )
    cv2.fillPoly(img, pts =[cmax], color=(255,255,255))
    cleaned_roi = img[:,:,0].astype(bool)
    return cleaned_roi




# --------------------------------        
# ----- reorder_h5py
# --------------------------------
def reorder_h5py(h5py_data,s):
    """h5py has a different ordering than what python does, so images are flipped. 
       this function reorders the data as expected. Requires a transpose and a conversion to numpy array.
       i.e. 2D data as (nrows,ncols)
       Inputs:
           - h5py_data:  h5py datatype for image arrays  
           - s:          slice number as an int  
       Returns:
           - ordered:     a dict pair of parameter and a nrow x ncol image (slice has been pre selected)"""
    ordered = dict()   
    for keyz in h5py_data.keys():
        ordered[keyz] = np.ascontiguousarray(h5py_data[keyz]).transpose()
        if ordered[keyz].ndim == 3:
            ordered[keyz] = ordered[keyz][:,:,s]
    return ordered



# --------------------------------        
# ----- loadHR_T2W
# --------------------------------
def loadHR_T2W(path, pnum, snum):
    """A function to determine whether the path is directed to the DECAES LWI data or NNLS and return
       the objects and the file itself.
       Inputs:
           - path:   a path for where the files are located (str)
           - pnum:   patient number (int)
           - snum:    slice number (int)
       Returns:
           - HR_img: a image 512x512 for the given patient and slice (np array)""" 
    hrpath = path + '/P*' +  str(pnum) + '*.mat'
    T2W_18 = loadmat(glob.glob(hrpath)[0])['T2W']
    HR_img = T2W_18[:,:,snum]
    return HR_img


# --------------------------------        
# ----- plotGif3D
# --------------------------------  
def plotGif3D(saveName, ax, anglesMulti, step=2):
    """ Creates gif through varying angles 
    Input:
        - saveName:    the name of the gif
        - ax:          the plot axis
        - anglesMulti: a 2D numpy array of the angles to be followed
        - step:        step size between each angle plotted
    """
    
    images = []
    # initializing the view
    ax.view_init(anglesMulti[0][0],anglesMulti[0][1])
    
    # saving gif
    if saveName is not None:
        # looping through each angle
        for i in range(len(anglesMulti)-1):
            angle1 = np.arange(anglesMulti[i][0],anglesMulti[i+1][0],2)
            angle2 = np.arange(anglesMulti[i][1],anglesMulti[i+1][1],2)

            numi = np.max([len(angle1),len(angle2)])
            if len(angle1) == 0:
                angle1 = np.ones(numi)*anglesMulti[i][0]
            if len(angle2) == 0:
                angle2 = np.ones(numi)*anglesMulti[i][1]

            # changing view and appending images
            for j in range(numi):
                ax.view_init(angle1[j],angle2[j])
                plt.savefig('saved_figure_' + '.png')
                images.append(imageio.imread('saved_figure_' + '.png'))
                os.remove('saved_figure_' + '.png')
            
            # saving image
            imageio.mimsave(saveName, images, fps=24)
    return


# --------------------------------        
# ----- printParamsAllROIs
# --------------------------------  
def printParamsAllROIs(T2DecayPath, echo, lwiPath, folder_roi, folder_pz, folder_gs, pnum, snum, parameters, cname='cubehelix', save = False, analysisType=None):
    """A function to print the slice and patient on screen
       Inputs:
            - T2DecayPath:  path to T2 decay data (str)     
            - echo:         the echo number to get image from (int)
            - lwiPath:      path to the LWI file folders (str)
            - folder_roi:   path to the full ROI file folders (str) 
            - folder_pz:    path to the PZ ROI files (str)
            - folder_gs:    path to the ROI files that malignancy levels based on GS (str)
            - snum:         slice number (int)
            - parameters:   the parameters selected for analysis e.g. LWF, sgm, mfr, etc. (list)
            - cname:        the seaborn colour palette to use
            - save:         to save or not save the file
            - analysisType: the type of analysis"""

    # loading T2 decay 
    T2Decay = loadmat(glob.glob(T2Decay_data_folder + '/P' + str(pnum).zfill(3) + '_cmplx*.mat')[0])
    T2Decay = T2Decay[list(T2Decay.keys())[3]] # usually always the third item in loadmat!

    # T2 decay
    img_decay=T2Decay[:,:,snum,echo] # the decay image

    # obtaining the fields and LWI dataset (decaes or NNLS) & the ROIs for the data and the malignant ROI
    fields, lwi, roi, roi_pz, roi_tz, gs_map = load_P_S(lwiPath, folder_roi, folder_pz, folder_gs, pnum, int(snum))
    
    # setting defaults for figure
    plt.rcParams['figure.figsize'] = (15,50) # set default size of plots
    plt.rcParams['image.cmap'] = 'gray'
    plt.figure()
    f,ax = plt.subplots(len(parameters)+1,2)
    
    # T2W image
    ax[0,0].imshow(img_decay)
    ax[0,0].set_title('T2W (' + str(np.size(img_decay,0)) + 'x' + str(np.size(img_decay,1)) + ')')
    ax[0,0].axis('off')
    
    # Getting GS and ROIs
#     colours = plt.cm.Set1(np.linspace(0,1,8))
#     colours = colours[:,0:3]
    colours = np.array(sns.color_palette(palette=cname, n_colors=8))
    # getting oultines
    pz_outline = outlineROI(roi_pz.astype(img_decay[0,0])).astype(bool)
    tz_outline = outlineROI(roi_tz).astype(img_decay[0,0]).astype(bool)
    baseim = np.dstack([(normalize(img_decay)),(normalize(img_decay)),(normalize(img_decay))]) # 3d image
    for c in range(6):
        baseim[(roi_pz|roi_tz)&(gs_map==c+1),:] = colours[c+2,:] # getting gs
    # making legend
    gs_labels = ['pz','tz','3+3','3+4','4+3','4+4','4+5','5+4']
    legend_dict=dict(zip(gs_labels,colours))
    #create patches
    patchList = []
    for key in legend_dict:
        data_key = mpatches.Patch(color=legend_dict[key], label=key)
        patchList.append(data_key)
    # putting roi ontop 
    baseim[pz_outline,:] = colours[0,:] # getting pz outline
    baseim[tz_outline,:] = colours[1,:] # getting tz outline
    ax[0,1].imshow(baseim)
    ax[0,1].set_title('Gleason Scores & zones')
    ax[0,1].axis('off')
    ax[0,1].legend(handles=patchList, fontsize='small')

    
    # Plotting parameters and overlaid gleason score
    for param in parameters: 
        # first image: image parameters
        lwi2 = copy.deepcopy(lwi) # deep copy to copy the image
        lwi2[param][~roi] = np.nan # remove all other parts of the image except ROI
        im1=ax[parameters.index(param)+1,0].imshow(lwi2[param])
        ax[parameters.index(param)+1,0].set_title('S:' + str(snum) + ' P' + str(pnum) + ', Param: ' + param, size=15)
        ax[parameters.index(param)+1,0].axis('off')
        plt.colorbar(im1, ax=ax[parameters.index(param)+1,0])
        # second image: overlaid with weight
        curIm = copy.deepcopy(lwi[param])
        curIm = np.dstack([normalize(curIm),normalize(curIm),normalize(curIm)])
        blankIm = np.zeros([np.size(img_decay,0),np.size(img_decay,1),3])
        for c in range(6):
            blankIm[(roi_pz|roi_tz)&(gs_map==c+1),:] = colours[c+2,:] # getting gs
        secondIm =cv2.addWeighted(blankIm,0.3,curIm,0.7,0.0)
        secondIm[~roi] = np.nan
        im2=ax[parameters.index(param)+1,1].imshow(secondIm)
        ax[parameters.index(param)+1,1].set_title('Showing Gleason Score. S:' + str(snum) + ' P' + str(pnum) + ', Param: ' + param, size=15)
        ax[parameters.index(param)+1,1].axis('off')
        ax[parameters.index(param)+1,1].legend(handles=patchList[2:], fontsize='small')
        ax[parameters.index(param)+1, 1].axis('off')

    if save:
        plt.savefig('P' + str(pnum)+ '_S'+ str(snum) + '_' + analysisType+ '.png')
    
    return 


def makeDir(bpath):
    """Makes a new, non-overwriting folder (makes a new folder name if name already exists), returns path of folder"""
    path=bpath
    pathMade=False
    numPath=1
    while not(pathMade):
        try:
            os.makedirs(path, exist_ok=False)
            pathMade = True
        except:
            path=bpath+str(numPath)
            numPath+=1
    return path

def init_excel(colNames, sheetName='Sheet1'):
    book = xlwt.Workbook()
    sheet = book.add_sheet(sheetName)
    for column, heading in enumerate(colNames):
        sheet.write(0, column, heading)
    return book,sheet

def excel_output(sheet, colNames, row2print):  
    nextRow = len(sheet._Worksheet__rows)
    
    for i,column in enumerate(colNames):
        sheet.write(nextRow, i, row2print[column])

    return sheet

In [ ]:
def getCompiledData(lwi, norm, dataCompile, benignCompile, maligCompile, roi_zone, gs_map, par, printGraphs=False, pnum=None,snum=None, ax=None, axNum=None, colours=None):    
    """ Compiles the data as pixels for all data, benign and malignant
    Inputs:
        - lwi
        - norm
        - dataCompile
        - benignCompile
        - maligCompile
        - roi_zone
        - gs_map
        - parameters
        - printGraphs=False
        - pnum
        - snum
        - ax=None
        - colours
    Returns:
        - dataCompile
        - benignCompile
        - oneMaligCompile """
    # Getting roiB and dictionary roiM
    roiB, roiMdict = splitGS_roi(roi_zone, gs_map)
    
    # For visualizing (1/3)
    if printGraphs:
        printIm = copy.deepcopy(lwi[par])
        printIm[~roiB] = np.nan
        # Getting GS and ROIs
        # getting oultines
        roi_outline = outlineROI(roi_zone.astype(np.float64)).astype(bool)
        baseim = np.dstack([(normalize(printIm)),(normalize(printIm)),(normalize(printIm))]) # 3d image
        for c in range(6):
            baseim[(roi_zone)&(gs_map==c+1),:] = colours[c+2,:] # getting gs
        # making legend
        gs_labels = ['pz','tz','3+3','3+4','4+3','4+4','4+5','5+4']
        legend_dict=dict(zip(gs_labels,colours))
        #create patches
        patchList = []
        for key in legend_dict:
            data_key = mpatches.Patch(color=legend_dict[key], label=key)
            patchList.append(data_key)
        # putting roi ontop 
        baseim[roi_outline,:] = colours[0,:] # getting pz outline
        ax[0,axNum].imshow(baseim)
        ax[0,axNum].set_title('P' + str(pnum) + ' S' + str(slice1) + ', Param: ' + par, size=15)
        ax[0,axNum].axis('off')
        ax[0,axNum].legend(handles=patchList, fontsize='small')
        ax[0,axNum].axis('off')
        # Histograms
        sns.histplot(lwi[par][roiB], color='dodgerblue', alpha=0.4,ax=ax[1,axNum],kde=True)
        for m in roiMdict:
            sns.histplot(lwi[par][roiMdict[m]], color = colours[m+2,:],alpha=0.4,ax=ax[1,axNum],kde=True)
        patchListHist = patchList[2:]
        patchListHist.append(mpatches.Patch(color='dodgerblue', label='benign'))
        ax[1,axNum].legend(handles=patchListHist,fontsize='small')

    # Normalizing data if chosen
    if norm == 'standardize':
        if par != 'N':
            lwi[par] = standardize(lwi[par])
    elif norm == 'normalize':
        if par != 'N':
            lwi[par] = normalize(lwi[par])
    elif norm == 'both':
        if par != 'N':
            lwi[par] = normalize(lwi[par])
            lwi[par] = standardize(lwi[par])
    tempDataCompile = copy.deepcopy(lwi[par])
    tempDataCompile[~roi_zone] = np.nan
    dataCompile[par] = np.concatenate((dataCompile[par],tempDataCompile[roi_zone]),axis=0)

    tempBenignCompile = copy.deepcopy(lwi[par])
    tempBenignCompile[~roiB] = np.nan
    benignCompile[par] = np.concatenate((benignCompile[par],tempBenignCompile[roi_zone]),axis=0)
    
    oneSliceMaligCompile = dict.fromkeys(np.arange(6),np.empty([0]))
    for m in roiMdict:
        tempMaligCompile = copy.deepcopy(lwi[par])
        tempMaligCompile[~roiMdict[m]] = np.nan
#         maligCompile[m][par] = np.concatenate((maligCompile[m][par],tempMaligCompile[roi_zone]),axis=0)
        oneSliceMaligCompile[m] = tempMaligCompile[roi_zone]
    return dataCompile, benignCompile, oneSliceMaligCompile


def splitGS_roi(roi2split, GSmap):
    """ Takes an incoming roi and splits it based on the GS score--> there are 6: '3+3','3+4','4+3','4+4','4+5','5+4
    Input:
        - roi2split: the roi [boolean]
        - GSmap:     the Gleason Score map with integer values of [0,6] where each int corresponds to a GS score: '0','3+3','3+4','4+3','4+4','4+5','5+4'
    Output:
        - roiB:    the benign roi [bool]
        - roiGS:   the rois split by GS as a dictionary. 0-->'3+3', 1--> '3+4', etc. [dict]"""
    
    # obtaining a copy of the 
    roi_r = copy.deepcopy(roi2split)
    # getting benign roi
    roiB = roi_r&~(GSmap>0)
    # getting malignant roi's split by GS
    roiGS = dict()
    for i in range(6):
        roiGS[i] = (roi_r)&(GSmap==i+1)
        
    return roiB, roiGS

In [ ]:

# --------------------------------        
# ----- load_single_patient_datacompile
# --------------------------------  
def load_single_patient_datacompile1(lwiPath, folder_roi, folder_pz, folder_gs, zone, pnum, slice1, parameters, norm=None, printGraphs=False, cname=None, bins=[100,100], save=False,analysisType=None):
    """ Loads the matlab or decaes matlab data and compiles all the data from one patient and its relevant slice files based on an roi. Provides the malignant/benign data
    Inputs:
        - lwiPath:      path to the LWI file folders (str)
        - folder_roi:   path to the ROI file folders (str) 
        - folder_pz:    path to the ROI files that pz (str)
        - folder_gs:    path to the GS mapping (str)
        - zone:         the data to analyze
        - pnum:         patient number (int)
        - slice1:       slice to analyze (int)
        - parameters:   the parameters selected for analysis e.g. LWF, sgm, mfr, etc. (list)
        - norm:         normalize the values before concatenating (either standardize, normalize, or both)
        - printGraphs:  prints the relevant graphs to visiualize the parameters and histogram of malig vs benign
        - cname:        the seaborn colour palette to use
        - bins:         Histogram bins
        - save:         to save the plots or not
        - analysisType: either 'decaes' or 'matlab'
    Returns:
        - dataCompile:   a dictionary with each parameter value as dictionaries and keys of column data: all pixel data given in the mask (roiPath) for each slice of a patient. Size: [#params, len of dataCompile]
        - labels:        a 1D array where 1 = malignant, 0 = benign
        - benignCompile: a dictionary with each parameter value as dictionaries and keys of column data: benign data given in the mask (~roiMaligPath&roiPath) for each slice of a patient. Size: [#params, len of dataCompile]
        - maligCompile:  a dictionary with each parameter value as dictionaries and keys of column data: benign data given in the mask (roiMaligPath&roiPath) for each slice of a patient. Size: [#params, len of dataCompile]"""
    %matplotlib inline
    patdirs = glob.glob(folder_roi+'/P*' + str(pnum) + '*') # getting the files that contain the patient number for the ROIs
    slices = [] # array that carries the slices available for the patient
    dataCompile = dict.fromkeys(parameters,np.empty([0]))
    benignCompile = dict.fromkeys(parameters,np.empty([0]))
    maligCompile = dict.fromkeys(parameters,dict.fromkeys(np.arange(6),np.empty([0])))
    # maligCompile = dict.fromkeys(parameters,np.empty([0]))
    binB = bins[0]
    binM = bins[1]

    # obtaining the fields and LWI dataset (decaes or NNLS) & obtaining the ROIs for the data and the malignant ROI

    fields, lwi, roi_all, roi_pz, roi_tz, gs_map = load_P_S(lwiPath, folder_roi, folder_pz, folder_gs, pnum, slice1)
    
    try:
        if zone =='full':
            roi_zone = roi_all
        elif zone == 'pz':
            roi_zone = roi_pz
        elif zone == 'tz':
            roi_zone = roi_tz
    except:
        print('Incorrect zone specified. Must be a subset of (full, "pz", or "tz"), not "' + zone + '".')
        raise()

    # For visualizing set-up (0/3)
    if printGraphs:
        plt.figure()
        plt.rcParams['figure.figsize'] = (24, 8) # set default size of plots
        f,ax = plt.subplots(2,len(parameters))
        colours = np.array(sns.color_palette(palette=cname, n_colors=8))
        # colour def 1
#         ccc = sns.husl_palette(9).as_hex()
#         colours = np.array([hex_to_rgb(ci.lstrip('#')) for ci in ccc])
        # colour def 2
#         colours = plt.cm.Set1(np.linspace(0,1,8))[:,0:3] # deciding colours of GS grading
#         colours = colours[:,0:3]
    else:
        colours=None
        ax = None

    # looping through all the various parameters to concatenate the "all", "benign", and "malignant" pixels
    for param in parameters:
        
        dataCompile, benignCompile, maligCompileTemp = getCompiledData(lwi, norm, dataCompile, benignCompile, maligCompile, roi_zone, gs_map, param, printGraphs,pnum,slice1,ax, int(parameters.index(param)-1),colours=colours)
        newVals = dict.fromkeys(np.arange(6),np.empty([0])) # empty key
        for m in maligCompile[param]: # compiling malig data
            values = copy.deepcopy(maligCompile[param][m])
            newVals[m] = np.concatenate((values,maligCompileTemp[m]),axis=0)
        maligCompile.update({param:newVals})
        # -----------------------------------------------------------------------------------------------------------
    # For visualizing (3/3)
    if printGraphs:
        plt.figure()
        plt.rcParams['figure.figsize'] = (20, 30) # set default size of plots
        f,ax = plt.subplots(len(parameters),1)
        for i,param in enumerate(parameters):
            # Histograms
            # -- benign
            sns.histplot(benignCompile[param], bins = binB, color='dodgerblue', alpha=0.4, ax=ax[i-1],element="step",kde=True)
            # -- malig
            for m in maligCompile[param]:
                sns.histplot(maligCompile[param][m], bins = binM, color = colours[m+2,:],alpha=0.4,ax=ax[i-1],element="step",kde=True)
            # making legend
            gs_labels = ['pz','tz','3+3','3+4','4+3','4+4','4+5','5+4']
            legend_dict=dict(zip(gs_labels,colours))
            patchList = [] #create patches
            for key in legend_dict:
                data_key = mpatches.Patch(color=legend_dict[key], label=key)
                patchList.append(data_key)
            patchListHist = patchList[2:]
            patchListHist.append(mpatches.Patch(color='dodgerblue', label='benign'))
            ax[i-1].legend(handles=patchListHist,fontsize='small')  
            # making title
            if norm == 'standardize' or norm == 'normalize' or norm == 'both':
                ax[i-1].set_title('(Norm type: '+ norm+ '), P' + str(pnum) + ' S' + str(slice1) +', Param: ' + param + ', Zone: ' + zone, size=15)
            else:
                ax[i-1].set_title('P' + str(pnum) + ', Param: ' + param + ', Zone: ' + zone, size=15)
        # saving data
        if save:
            if norm == 'standardize' or norm == 'normalize' or norm == 'both':
                plt.savefig('P' + str(pnum)+ '_S' + str(slice1) + '_' + zone+ '_' + analysisType+ '_hist_' + norm + '.png')
            else:
                plt.savefig('P' + str(pnum)+ '_S' + str(slice1) + '_' + zone+ '_' + analysisType+ '_hist.png')
    if printGraphs is False:
        plt.close()

    return dataCompile, benignCompile, maligCompile


def hex_to_rgb(hex):
    rgb = []
    hex = hex[1:]
    for i in (0, 2, 4):
        decimal = int(hex[i:i+2], 16)/255
        rgb.append(decimal)
    return rgb

### Functions w/ Calculations

In [ ]:
# --------------------------------        
# ----- sigmoid
# --------------------------------  
def sigmoid(x): 
    """  Compute the sigmoid of x
    Input:
        x -- A scalar or numpy array of any size.
    Return:
        s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

# --------------------------------        
# ----- tanh
# --------------------------------  
def tanh(x): 
    """  Compute the tanh function of 
    Input:
        x -- A scalar or numpy array of any size.
    Return:
        t -- tanh(x)
    """
    t = np.tanh(x)
    return t

# --------------------------------        
# ----- normalize
# --------------------------------  
def normalize(M, normRange=None):
    """ Normalize matrix M based to exist in [0,1] and removes nan's from calculation. Returns a matrix
    Inputs:
        - M:     matrix of size [n,m]
        - normRange: a an array of size 1x2 that contains the max and min range [min, max]
    Outputs:
        - Mnorm: normalized matrix of size [n,m]"""
    Mcopy = M[~np.isnan(M)]        
    Mnorm = (M - np.min(Mcopy)) / (np.max(Mcopy) - np.min(Mcopy))
    
    if normRange is not None:
        Mnorm = Mnorm*(normRange[1] - normRange[0]) + normRange[0]
        
    return Mnorm

# --------------------------------        
# ----- standardize
# --------------------------------  
def standardize(M):
    """ Standardize matrix M and removes nan's from calculation. Returns normalization as a vector
    Inputs:
        - M:    matrix of size [n,m]
    Outputs:
        - Mstd: normalized matrix of size [n,m]"""
    Mcopy = M[~np.isnan(M)]  
    Mstd = (M - np.mean(Mcopy)) / np.std(Mcopy)
    return Mstd



# --------------------------------        
# ----- outlineROI
# --------------------------------  
def outlineROI(mask):
    # getting dilated outline of mask
    kernSize = 2
    kernel = np.ones((kernSize,kernSize), np.uint8)
    img_outline = cv2.dilate(np.uint8(mask),kernel,iterations=1)-cv2.erode(np.uint8(mask),kernel,iterations=1)
    return img_outline


# --------------------------------        
# ----- findTopPlotParams
# --------------------------------        
def findTopPlotParams(benComp, malComp, parameters, show=False):
    """ Finds the top 2 and top 3 parametesr to plot based on their largest differences (sigmoid + standrdization)
    Inputs:
    - benComp:    the benign component dictionary dataset
    - malComp:    the malignant component dictionary dataset
    - parameters: the string array of parameter names
    - show:       to show the figures or not
    Outputs:
    - pltParams2: the top 2 parameters with the largest difference in mal/ben [string array]
    - pltParams3: the top 3 parameters with the largest difference in mal/ben [string array]"""
    diffs = np.zeros(len(parameters))
    for i,param in enumerate(parameters):
        if ~np.isclose(np.std(malComp[param][~np.isnan(malComp[param])]),0): 
            b = sigmoid((standardize(benComp[param][~np.isnan(benComp[param])])))
            m =  sigmoid((standardize(malComp[param][~np.isnan(malComp[param])])))
        else:
            b = sigmoid(benComp[param][~np.isnan(benComp[param])])
            m = sigmoid(malComp[param][~np.isnan(malComp[param])])
        diffs[i] = np.abs(np.mean(b) - np.mean(m))
        if show: 
        # Plotting
            plt.figure()
            sns.histplot(b, bins = 300, color='dodgerblue', alpha=0.4)
            sns.histplot(m, bins = 100, color = 'coral',alpha=0.4)
            plt.title(param)
    if show:
        plt.figure()
        plt.plot(np.arange(0,7),diffs,'x')
    
    pltParams2 = ['','']
    pltParams3 = ['','','']
    pltParams2[0] = parameters[np.argmax(diffs)]
    diffs[np.argmax(diffs)] = -1
    pltParams2[1] = parameters[np.argmax(diffs)]
    diffs[np.argmax(diffs)] = -1
    pltParams3[0] = pltParams2[0]
    pltParams3[1] = pltParams2[1]
    pltParams3[2] = parameters[np.argmax(diffs)]
    
    return pltParams2, pltParams3

# --------------------------------        
# ----- createXforPCA
# --------------------------------  
def createXforPCA(benComp, malComp, parameters):
    """ Finding the default valid-pixel range length based on parameters given. Looks for smallest common denominator
    Input:
    - benComp:    the benign component dictionary dataset
    - malComp:    the malignant component dictionary dataset
    - parameters: the string array of parameter names
    
    Returns:
    - X:   The concatenated matrix
    """
    largestLen = np.zeros(len(parameters))
    for i, param in enumerate(parameters): 
        largestLen[i] = len(benComp[param][~np.isnan(benComp[param])])+len(malComp[param][~np.isnan(malComp[param])])
    default_pixel_length_param = parameters[np.argmin(largestLen)]
    print('Minimum of set is the default valid-pixel range: ' + str((np.unique(largestLen))))

    # Creating X matrix for SVD
    X = np.zeros((int(largestLen[np.argmin(largestLen)]),len(parameters))) # initializing X matrix
    labels = np.zeros((int(largestLen[np.argmin(largestLen)]),1)) #creating labels where 1 = cancer, 0 = benign
    for i, param in enumerate(parameters): # X is a matrix of [n,p] n = #total pixels, p = # parameters. The first part of n is malignant, second part of n is benign
        X[:,i] = (np.concatenate((malComp[param][~np.isnan(malComp[default_pixel_length_param])], benComp[param][~np.isnan(benComp[default_pixel_length_param])]),axis=0))
    return X

# --------------------------------        
# ----- returnPCa_matrix
# --------------------------------      
def returnPCa_matrix(numPC, X, VT, labels):
    """ Calculates the projection of matrix X in pricinple components and returns the benign and malignant data
    Inputs:
    - numPC:  the number of principal components to include
    - X:      the original matrix
    - VT:     the matrix containing principal axis
    - labels: an array containing 1 = malignant, 0 = benign
    
    Outputs:
    - X_pca: the matrix X projected in #numPC axis
    - ben:   the portion of X_pca attributed to benign points
    - mal:   the portion of X_pca attributed to malignant points"""
    X_pca = np.zeros((len(labels),numPC))
    newRow = np.zeros((1,numPC))

    # looping through finding the principal components
    for j in range(X.shape[0]):
        for k in range(numPC):
            X_pca[j,k] = np.dot(VT[k,:],X[j,:].T)
    
    # denoting which is benign and malignant        
    ben = X_pca[int(sum(labels==1))::,:] # second part of data is benign
    mal = X_pca[0:int(sum(labels==1))-1,:] # first part of data is malignant
    
    return X_pca, ben, mal

def showSepHist(dat, param, roiB, roiMdict, ax1,ax2, leg, colours,dictkey,gs_labels):
    bflag = True            
    #    -- malig
    for m in roiMdict:
        dictkey[toSave[m+8]] = None
        dictkey[toSave[m+14]]  = None
        dictkey[toSave[m+20]]  = None
        if np.sum(dat[roiMdict[m]][~np.isnan(dat[roiMdict[m]])]).astype(bool) & (np.sum(roiMdict[m])>15):
            # setting range of histograms
            binn = np.round((1 + 3.322*np.log(len(dat[roiB])+len(dat[roiMdict[m]])))).astype(int)
            nnan1 = dat[roiB]; nnan1 = nnan1[~np.isnan(nnan1)]
            nnan2 = dat[roiMdict[m]]; nnan2 = nnan2[~np.isnan(nnan2)]
            ran = [min(np.min(nnan1),np.min(nnan2)), max(np.max(nnan1),np.max(nnan2))] 
            if nnan2 is float("inf"):
                ran = None
                print('inf data. ')
            if bflag:
                ax = sns.histplot(dat[roiB], bins = binn, ax=ax1, color='dodgerblue', alpha=0.4,element="step",binrange = ran,kde=True)
                ax2 = sns.ecdfplot(dat[roiB], ax=ax2, color='dodgerblue')
                bflag = False
            ax = sns.histplot(dat[roiMdict[m]], bins = binn , ax=ax1, color = colours[m+2,:],alpha=0.4,element="step",binrange = ran,kde=True)
            ax2 = sns.ecdfplot(dat[roiMdict[m]], ax=ax2, color = colours[m+2,:])
            
            # counting overlap percentage
            countB,binB = np.histogram(dat[roiB], bins=binn, range=ran)
            countM,binM = np.histogram(dat[roiMdict[m]], bins=binn, range=ran)
            ol = (np.sum(np.minimum(countB,countM)))
            pol = ol/(np.sum(countM))
            dictkey[toSave[m+8]] = pol.astype(float)
            
            # Get the two lines from the axes to generate shading
            l1 = ax.lines[0]
            l2 = ax.lines[-1]
            # Get the xy data from the lines so that we can shade
            x1, y1 = l1.get_xydata().T
            x2, y2 = l2.get_xydata().T
            # Calculating overlap
            xmin = max(x1.min(), x2.min())
            xmax = min(x1.max(), x2.max())
            x = np.linspace(xmin, xmax, 1000)
            y1a = np.interp(x, x1, y1)
            y2a = np.interp(x, x2, y2)
            y = np.minimum(y1a, y2a)
#             axx.fill_between(x, y, color="red", alpha=0.3)
#             plt.plot(x,y)
            area = trapz(y, dx=(x[-1]-x[0])/len(x))
            perc = area / trapz(y2a, dx=(x[-1]-x[0])/len(x)) 
            dictkey[toSave[m+14]] = perc.astype(float)
            
            # Calculating inverted ROC for prediction intensity 1 == cancer
            # if (param == 'N') or (param=='A2') or (param=='LWF') or (param=='gmT2') or (param=='T21'):
            d1 = np.abs(1-dat[roiB]/max(np.max(dat[roiB]),np.max(dat[roiMdict[m]])))
            d2 = np.abs(1-dat[roiMdict[m]]/max(np.max(dat[roiB]),np.max(dat[roiMdict[m]])))
            # else:
            #     d1 = dat[roiB]/max(np.max(dat[roiB]),np.max(dat[roiMdict[m]]))
            #     d2 = dat[roiMdict[m]]/max(np.max(dat[roiB]),np.max(dat[roiMdict[m]]))
            fpr, tpr, thresholds = roc_curve(np.concatenate([np.ones(np.shape(d2)), np.zeros(np.shape(d1))]), np.concatenate([d2,d1]))
            roc_auc = auc(fpr, tpr)
            dictkey[toSave[m+20]] = roc_auc.astype(float)
#             plot_roc_curve(fpr, tpr, roc_auc,ccc=colours[m+2,:],lname=gs_labels[m+2],ax=ax nm32)
    ax1.legend(handles=leg,fontsize='small');
    return dictkey

 
# --------------------------------        
# ----- showSepHisto
# --------------------------------      
def showSepHisto(dat, param, roiB, roiMdict, ax, leg, colours):
    sns.histplot(dat[roiB], bins = np.round(1 + 3.322*np.log(len(dat[roiB]))).astype(int), ax=ax, color='dodgerblue', alpha=0.4,element="step",kde=True)
    #    -- malig
    for m in roiMdict:
        if np.sum(dat[roiMdict[m]][~np.isnan(dat[roiMdict[m]])]).astype(bool):
            sns.histplot(dat[roiMdict[m]], bins = np.round(1 + 3.322*np.log(len(dat[roiMdict[m]]))).astype(int), ax=ax, color = colours[m+2,:],alpha=0.4,element="step",kde=True)
    ax.legend(handles=leg,fontsize='small')   
    
# --------------------------------        
# ----- plot_roc_curve
# --------------------------------   
def plot_roc_curve(fpr, tpr, roc_auc, lw=2,ccc=None,ax=None,lname='None'):
    """Plot roc curve"""
    lw = lw
    ax.plot(fpr, tpr, color=ccc,
             lw=lw, label= lname + ', AUC=%0.2f' % roc_auc)
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")    

# --------------------------------        
# ----- getOutlineRoi
# -------------------------------- 
def getOutlineRoi(mask, k=2):
    """ Gets the outline of a binary mask 
    Input:
        - mask: the binary mask
        - k:    the kernel size
    Return:
        - mask_outline: returns a binary mask"""
    # Specifying kernel size
    kernel = np.ones((k,k), np.uint8)
    mask = np.uint8(mask)
    img_erosion = cv2.erode(mask, kernel, iterations=1)
    img_mod = cv2.dilate(img_erosion,kernel,iterations=1)
    mask_outline = cv2.dilate(img_mod,kernel,iterations=1)-cv2.erode(img_mod,kernel,iterations=1)
    return mask_outline


# Testing functions

#### Working Fn's

In [ ]:
def runAnalysisWeightingGSmap(save, foldersavep, sheet, row2print, figShow, T2DecayPath, echo, lwipath, folder_roi, folder_pz, folder_gs, folder_HR, pnum, snum, norm, weights=1, cname='cubehelix'):
    """To show the image with weighting
    Inputs:
        - save:         whether or not to save figures 
        - foldersavep:
        - row2print: 
        - figShow:      whether or not to show figures
        - T2DecayPath:  path to T2 decay data (str)
        - echo:         the echo number to get image from (int)
        - lwiPath:      path to the LWI file folders (str)
        - folder_roi:   path to the full ROI file folders (str) 
        - folder_pz:    path to the PZ ROI files (str)
        - folder_gs:    path to the ROI files that malignancy levels based on GS (str)
        - folder_HR:    path to the high-res T2W matlab files that are 512x512x18 (str)
        - pnum:         the patient numbers (int array)
        - snum:         specifying a slice for single pat + single slice analysis (int)
        - norm:         normalize the values before concatenating (either standardize, normalize, or both)
        - weighta:      weighting [array or int in []] on images where Im = normalize(Im*(1-weighting) + overlay*weighting)
        - cname:        for the legend and viewing data
    Returns:
        - book:         an excel spreadsheet of the data collected
        """
    # Settings for printing (individual pictures each)
    plt.rcParams['figure.figsize'] = (10,10) # set default size of plots
    plt.rcParams['image.cmap'] = 'gray'
    
    T2Decay = loadmat(glob.glob(T2DecayPath + '/P' + str(pnum).zfill(3) + '_cmplx*.mat')[0])
    T2Decay = T2Decay[list(T2Decay.keys())[3]] # usually always the third item in loadmat!
    
    parameters,lwi = loadDecaesOrMatlab(lwiPath, pnum, snum)
    roi_all, roi_pz, roi_tz, gs_map = loadROIall(folder_roi,folder_pz,folder_gs,pnum,snum)
    zones = ['pz','tz','full','full_altWeight']
    roi_zones=dict()
    roi_zones['pz'] = roi_pz
    roi_zones['tz'] = roi_tz
    roi_zones['full'] = roi_all
    
#     roi[np.where(np.isclose(lwi['LWF'], 0))] = False
#     roiM[np.where(np.isclose(lwi['LWF'], 0))] = False
    
    # Getting the ROIs of pz and tz
    roiB=dict()
    roiMdict=dict()
    roi_t2w_save=dict()
    roiB['pz'], roiMdict['pz'] = splitGS_roi(roi_pz, gs_map)
    roiB['tz'], roiMdict['tz'] = splitGS_roi(roi_tz, gs_map)
    roiB['full'], roiMdict['full'] = splitGS_roi(roi_all, gs_map)
    
    # T2 decay
    img_decay=T2Decay[:,:,snum,echo] # the decay image
    
    
    # Fig1 = showing image decay
    plt.figure()
    plt.imshow(img_decay)
    plt.title('T2 Decay (' + str(np.size(img_decay,0)) + 'x' + str(np.size(img_decay,1)) + '). P' +str(pnum) + ' S' +str(snum)+ ' E' + str(echo) +'.' )
    plt.axis('off')
    if save:
        plt.savefig(foldersavep +'/'+'P'+str(pnum) + '_S' +str(snum) +'_T2D.png')
    if figShow:
        plt.show()
    
    # Fig2 = showing malignancy
    # setting defaults for figure
    plt.figure()        
    colours = np.array(sns.color_palette(palette=cname, n_colors=8))
    # making legend
    gs_labels = ['pz','tz','3+3','3+4','4+3','4+4','4+5','5+4']
    legend_dict=dict(zip(gs_labels,colours))
    patchList = []    #create patches
    patchList.append(mpatches.Patch(color=legend_dict[gs_labels[0]], label=gs_labels[0]) ) #roi pz
    patchList.append(mpatches.Patch(color=legend_dict[gs_labels[1]], label=gs_labels[1]) ) #roi tz
    # getting oultines
    pz_outline = outlineROI(roi_pz.astype(img_decay[0,0])).astype(bool)
    tz_outline = outlineROI(roi_tz).astype(img_decay[0,0]).astype(bool)
    baseim = np.dstack([(normalize(img_decay)),(normalize(img_decay)),(normalize(img_decay))]) # 3d image
    for c in range(6):
        baseim[(roi_pz|roi_tz)&(gs_map==c+1),:] = colours[c+2,:] # getting gs
        if (np.sum(baseim[(roi_pz|roi_tz)&(gs_map==c+1),:])).astype(bool):
            data_key = mpatches.Patch(color=legend_dict[gs_labels[c+2]], label=gs_labels[c+2]) 
            patchList.append(data_key)
    # putting roi below all the grading 
    baseim[pz_outline,:] = colours[0,:] # getting pz outline
    baseim[tz_outline,:] = colours[1,:] # getting tz outline
    
    plt.imshow(baseim)
    plt.title('Gleason Grading & Zones')
    plt.axis('off')
    plt.legend(handles=patchList, fontsize='small');
    if save:
        plt.savefig(foldersavep +'/'+'P'+str(pnum) + '_S' +str(snum) +'_GSmap.png')
    if figShow:
        plt.show()
    else:
        plt.close()
    
    # #################################
    # ###########
    # Getting the PCA image
    # getting the weighting image inside just the full ROI
    lwi_PCA = dict()
    lwi_nonan = copy.deepcopy(lwi)
    # for param in parameters:
    #     lwi_nonan[param][~roi_all] = np.nan
    origshape = np.shape(lwi_nonan[parameters[0]])
    X_lwi = np.asarray((lwi_nonan[parameters[0]].flatten()))
    pca_mean = np.mean(X_lwi[~(np.isnan(X_lwi))], axis=0)
    for i,parami in enumerate(parameters[1:]):
        X_temp = np.asarray(standardize(lwi_nonan[parami]).flatten())
        X_lwi = np.vstack((X_lwi,X_temp))
        pca_mean = np.append(pca_mean,np.mean(X_temp[~(np.isnan(X_temp))],axis=0))
           
    X_lwi = X_lwi.T
    pca_transform_mat = dict()
    pca_transform_mat['pz'] = np.array([[-0.47973739,  0.49448727,  0.4955605 , -0.04783564,  0.44775352, 0.26099344,  0.09415476]])
    pca_transform_mat['tz'] = np.array([[-0.4510,    0.5161,    0.5176,    0.0954,    0.4221,    0.2315,    0.1464]])
    pca_transform_mat['full'] = np.array([[-0.485654, 0.505522, 0.507955, 0.026392, 0.423345, 0.227122, 0.138086 ]])
    # performing transformation
    lwi_PCA['pz']= np.reshape(np.dot(X_lwi - pca_mean, pca_transform_mat['pz'].T), origshape)
    lwi_PCA['tz']= np.reshape(np.dot(X_lwi - pca_mean, pca_transform_mat['tz'].T), origshape)
    lwi_PCA['full']= np.reshape(np.dot(X_lwi - pca_mean, pca_transform_mat['pz'].T), origshape)
    
    # ###########
    # ####################################
    
    # looping through weights:
    firstweightflag = 0
    
    try:
        for weighting in weights:
            weightPath = makeDir(foldersavep+'/W_'+str(weighting))
            hr_full_savePZ = dict()
            for zonei, zone in enumerate(zones):
                # ---- adding directory for a zone (new folder)
                zonePath = makeDir(weightPath+'/'+zone)

                if zonei == 3: # this means we're in iterating to the "full" roi
                    zone = 'tz'
                    beforesave = save
                    save = False

                # ---- plot & histogram
                roi_zone = roi_zones[zone]
                row2print['Patient'] = float(pnum)
                row2print['Slice'] = float(snum)
                row2print['Echo'] = echo
                row2print['Norm?'] = norm
                row2print['Weighting %'] = float(weighting)
                row2print['Zone'] = zone


                # the malig rois split by GS as a dictionary. 0-->'3+3', 1--> '3+4', etc. [dict]
                dataCompile, benignCompile, maligCompile = load_single_patient_datacompile1(lwiPath, folder_roi, folder_pz, folder_gs, zone, pnum, snum, parameters, norm=norm, printGraphs=False,save=False)

                # weighted image
                param = 'PCA(std)'

                # getting the weighting image inside just the full ROI
                img2 = copy.deepcopy(lwi_PCA[zone])
                if np.isnan(np.sum((img2[roi_zone]))):
                    roi_zone[np.isnan(img2*roi_zone)] = False
                img2[~roi_zone] = np.nan # removing background!

                # required for scaling of image properly when weighting
                tempIm = copy.deepcopy(img_decay)
                tempIm = (tempIm - min(img_decay[roi_zone]))/(max(img_decay[roi_zone])-min(img_decay[roi_zone]))            

                # Obtaining weighting images
                # --- Tanh on Standardized
                w_tstand = normalize(img_decay)
                w_temp = tanh(standardize(img2))    
                new_w_temp = tempIm*(1-weighting) + normalize(tempIm*w_temp)*weighting
                new_w_temp = normalize(new_w_temp, [min(w_tstand[roi_zone]),max(w_tstand[roi_zone])])
                w_tstand[roi_zone] = new_w_temp[roi_zone]

                # --- sigmoid on Standardized
                w_sstand = normalize(img_decay)
                w_temp = sigmoid(standardize(img2))    
                new_w_temp = tempIm*(1-weighting) + normalize(tempIm*w_temp)*weighting
                new_w_temp = normalize(new_w_temp, [min(w_tstand[roi_zone]),max(w_tstand[roi_zone])])
                w_sstand[roi_zone] = new_w_temp[roi_zone]

                # Showing figures
                plt.rcParams['figure.figsize'] = (30,20) # set default size of plots
                plt.figure()
                f,ax = plt.subplots(3,4)
                ax[0,0].imshow(img2)
                ax[0,0].set_title('Zone: ' + zone + ', Param: ' +param)
                ax[0,0].axis('off')

                ax[0,1].imshow(img_decay)
                ax[0,1].set_title('T2W image')
                ax[0,1].axis('off')

                ax[0,2].imshow(w_tstand)
                ax[0,2].set_title('Tanh + Standardized')
                ax[0,2].axis('off')

                ax[0,3].imshow(w_sstand)
                ax[0,3].set_title('Sigmoid + Standardized')
                ax[0,3].axis('off')
                #^------------------------------------^  
                # ########################################
                ###########
                if firstweightflag == 0:
                    firstweightflag = 1
                    # Call to Matlab to get high-res images
                    # Note: the matlab function needs to be in the same folder as the working folder of the python script being run
                    # output of the function is a "struct" with two variables 't2w_roi' and 't2w_slice'
                    eng = matlab.engine.start_matlab()
                    matlab_struct = eng.fn_runApp(matlab.double(int(pnum)), matlab.double(int(snum)), matlab.double(roi_all.tolist()), matlab.double(lwi_PCA[zone].tolist()), nargout = 1)
                    if len(matlab_struct['t2w_roi'])==0:
                        raise NameError('Cancelled')
                    
                    # getting the high-res ROI and silce
                    hr_roi_full = np.array(matlab_struct['t2w_roi'])
                    hr_slice = int(matlab_struct['t2w_slice'])-1
                    imweight = matlab_struct['weight']
                    # high-res T2W images
                    hr_img = loadHR_T2W(folder_HR, pnum, hr_slice)
                    orig_hr_img = copy.deepcopy(hr_img)
                    # saving figure
                    imageio.imwrite(foldersavep +'/P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_slice_'+ str(hr_slice+1) + '.png', np.uint8(hr_roi_full)*255)
                    imageio.imwrite(foldersavep +'/P'+str(pnum)+'_S' +str(snum)+'_T2W_slice_'+ str(hr_slice+1) + '.png', np.uint8(hr_img/hr_img.max()*255))

                    # getting points of the small rectangle of roi (rx/ry) and larger roi (nx/ny)
                    rx1 = int(matlab_struct['dimSmall']['x1'])-1
                    rx2 = int(matlab_struct['dimSmall']['x2'])
                    ry1 = int(matlab_struct['dimSmall']['y1'])-1
                    ry2 = int(matlab_struct['dimSmall']['y2'])
                    nx1 = np.min(np.where(hr_roi_full)[0])
                    nx2 = np.max(np.where(hr_roi_full)[0])+1
                    ny1 = np.min(np.where(hr_roi_full)[1])
                    ny2 = np.max(np.where(hr_roi_full)[1])+1

                    hr_subroi=dict()
                    hr_subroi['full'] = ski_resize(roi_all[rx1:rx2, ry1:ry2], [nx2-nx1,ny2-ny1], anti_aliasing=False)
                    hr_subroi['pz'] = ski_resize(roi_pz[rx1:rx2, ry1:ry2], [nx2-nx1,ny2-ny1], anti_aliasing=False)
                    hr_subroi['tz'] = ski_resize(roi_tz[rx1:rx2, ry1:ry2], [nx2-nx1,ny2-ny1], anti_aliasing=False)


                    hr_roi=dict()
                    hr_roi['pz'] = np.zeros(np.shape(hr_img))
                    hr_roi['tz'] = np.zeros(np.shape(hr_img))
                    hr_roi['full'] = hr_roi_full
                    hr_roi['pz'][nx1:nx2,ny1:ny2] = hr_subroi['pz']
                    hr_roi['tz'][nx1:nx2,ny1:ny2] = hr_subroi['tz']
                    hr_roi['full'] = hr_roi['full']>0
                    hr_roi['pz']   = hr_roi['pz']>0
                    hr_roi['tz']   = hr_roi['tz']>0
                else:
                    hr_img = copy.deepcopy(orig_hr_img)
                    
                # --- PCA on T2W High-Res 512x512 images
                if zonei == 3:
                    hr_img = copy.deepcopy(hr_full_savePZ[param])
                tempIm_hr = copy.deepcopy(hr_img)
                tempIm_hr = (tempIm_hr - min(hr_img[hr_roi[zone]]))/(max(hr_img[hr_roi[zone]])-min(hr_img[hr_roi[zone]]))
                # obtaining the PCA image
                img2 = copy.deepcopy(lwi_PCA[zone])
                if np.isnan(np.sum((img2[roi_zone]))):
                    roi_zone[np.isnan(img2*roi_zone)] = False 
                # not removing nans from pca image 
                ssub_im = ski_resize(img2[rx1:rx2, ry1:ry2], [nx2-nx1,ny2-ny1], anti_aliasing=False)
                clippedROIt2w = hr_subroi[zone]>0
                ssub_im[~clippedROIt2w] = np.nan

                hr_w_tstand = normalize(hr_img)
                w_temp_hr_sub = tanh(standardize(ssub_im))    
                new_w_temp = tempIm_hr[nx1:nx2,ny1:ny2]*(1-weighting) + normalize(tempIm_hr[nx1:nx2,ny1:ny2]*w_temp_hr_sub)*weighting
                new_w_temp = normalize(new_w_temp, [min(hr_w_tstand[hr_roi[zone]]),max(hr_w_tstand[hr_roi[zone]])])
                hr_overlay_sub = hr_w_tstand[nx1:nx2,ny1:ny2]
                hr_overlay_sub[clippedROIt2w] = new_w_temp[clippedROIt2w]
                hr_w_tstand[nx1:nx2,ny1:ny2] = hr_overlay_sub
                if zone == 'pz':
                    hr_full_savePZ[param] = copy.deepcopy(hr_w_tstand) # making a copy to save in pz and then combine with tz later for the "full"
                
                if figShow:
                    plt.figure()
                    plt.rcParams['figure.figsize'] = (10,10) # set default size of plots
                    plt.imshow(np.array(matlab_struct['outputHR']))
                else:
                    plt.close()
                # imageio.imwrite(zonePath +'/Overlay_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_'+zone+'_slice_'+ str(hr_slice+1) + '_w' +str(weighting) + '_PCA.png', (np.array(matlab_struct['outputHR'])*255).astype(np.uint8))
                if zonei == 3:
                        imageio.imwrite(zonePath +'/Overlay_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_full_slice_'+ str(hr_slice) + '_w' +str(weighting) + '_' + param +'.png', (hr_w_tstand*255).astype(np.uint8))
                else:
                    imageio.imwrite(zonePath +'/Overlay_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_'+zone+'_slice_'+ str(hr_slice) + '_w' +str(weighting) + '_' + param +'.png', (hr_w_tstand*255).astype(np.uint8))

                ###########
                # ########################################
                #^------------------------------------^  

                # analysis
                # Analysis of weighted - Histograms ------------------------------------,
                # making legend & colours
                colours = np.array(sns.color_palette(palette=cname, n_colors=8))
                patchList = [] #create patches
                patchList.append(mpatches.Patch(color='dodgerblue', label='Benign'))
                for m in maligCompile['LWF']:
                    if np.sum(maligCompile['LWF'][m][~np.isnan(maligCompile['LWF'][m])]).astype(bool):
                        data_key = mpatches.Patch(color=legend_dict[gs_labels[m+2]], label=gs_labels[m+2]) 
                        patchList.append(data_key)

                if zonei < 3:
                    row2print['Param'] = param
                    # --- Original T2W
                    row2print = showSepHist(img_decay,param,roiB[zone],roiMdict[zone],ax[1,1], ax[2,1],patchList, colours,row2print,gs_labels)
                    row2print['Weighting fn'] = 'Original'
                    sheet = excel_output(sheet,toSave,row2print)

                    # --- Tanh on Standardized
                    row2print = showSepHist(w_tstand,param,roiB[zone],roiMdict[zone],ax[1,2], ax[2,2],patchList, colours,row2print,gs_labels)
                    row2print['Weighting fn'] = 'Tanh'
                    sheet = excel_output(sheet,toSave,row2print)

                    # --- sigmoid on Standardized
                    row2print = showSepHist(w_sstand,param,roiB[zone],roiMdict[zone],ax[1,3], ax[2,3],patchList, colours,row2print,gs_labels)
                    row2print['Weighting fn'] = 'Sigmoid'
                    sheet = excel_output(sheet,toSave,row2print)
                    
                    ax[1,0].imshow(baseim)
                    ax[1,0].set_title('Gleason Grading & Zones')
                    ax[1,0].axis('off')
                    ax[1,0].legend(handles=patchList, fontsize='small');
                    f.delaxes(ax[2,0])
                if save:
                    imageio.imwrite(zonePath +'/LR_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_'+zone+'_w' +str(weighting) + '_' + param +'.png', (w_tstand*255).astype(np.uint8))
                    # plt.savefig(zonePath +'/'+'P'+str(pnum) + '_S' +str(snum) + '_'+param+'_w' +str(imweight) + '_' + zone + '.png')
                if figShow:
                    plt.show()
                else:
                    plt.close()


                # printing out LWI parameter values
                for i,param in enumerate(parameters):       

                    # weighted image
                    overlayIm = lwi[param]

                    # getting the weighting image inside just the full ROI
                    img2=copy.deepcopy(overlayIm) 
                    if np.isnan(np.sum((img2[roi_zone]))):
                        roi_zone[np.isnan(img2*roi_zone)] = False
                    img2[~roi_zone] = np.nan # removing background!

                    # required for scaling of image properly when weighting
                    tempIm = copy.deepcopy(img_decay)
                    tempIm = (tempIm - min(img_decay[roi_zone]))/(max(img_decay[roi_zone])-min(img_decay[roi_zone]))

                    # Obtaining weighting images
                    # --- Tanh on Standardized
                    w_tstand = normalize(img_decay)
                    w_temp = tanh(standardize(img2))    
                    new_w_temp = tempIm*(1-weighting) + normalize(tempIm*w_temp)*weighting
                    new_w_temp = normalize(new_w_temp, [min(w_tstand[roi_zone]),max(w_tstand[roi_zone])])
                    w_tstand[roi_zone] = new_w_temp[roi_zone]

                    # --- sigmoid on Standardized
                    w_sstand = normalize(img_decay)
                    w_temp = sigmoid(standardize(img2))    
                    new_w_temp = tempIm*(1-weighting) + normalize(tempIm*w_temp)*weighting
                    new_w_temp = normalize(new_w_temp, [min(w_tstand[roi_zone]),max(w_tstand[roi_zone])])
                    w_sstand[roi_zone] = new_w_temp[roi_zone]

                    # Showing figures
                    plt.rcParams['figure.figsize'] = (30,20) # set default size of plots
                    plt.figure()
                    f,ax = plt.subplots(3,4)
                    ax[0,0].imshow(img2)
                    ax[0,0].set_title('Zone: ' + zone + ', Param: ' +param)
                    ax[0,0].axis('off')

                    ax[0,1].imshow(img_decay)
                    ax[0,1].set_title('T2W image')
                    ax[0,1].axis('off')

                    ax[0,2].imshow(w_tstand)
                    ax[0,2].set_title('Tanh + Standardized')
                    ax[0,2].axis('off')

                    ax[0,3].imshow(w_sstand)
                    ax[0,3].set_title('Sigmoid + Standardized')
                    ax[0,3].axis('off')
                    #^------------------------------------^  
                    # --- PCA on T2W High-Res 512x512 images
                    # high-res T2W images
                    if zonei == 3:
                        hr_img = copy.deepcopy(hr_full_savePZ[param])
                    tempIm_hr = copy.deepcopy(hr_img)
                    tempIm_hr = (tempIm_hr - min(hr_img[hr_roi[zone]]))/(max(hr_img[hr_roi[zone]])-min(hr_img[hr_roi[zone]]))
                    # obtaining the PCA image
                    img2 = copy.deepcopy(overlayIm)
                    if np.isnan(np.sum((img2[roi_zone]))):
                        roi_zone[np.isnan(img2*roi_zone)] = False 
                    # not removing nans from pca image 
                    ssub_im = ski_resize(img2[rx1:rx2, ry1:ry2], [nx2-nx1,ny2-ny1], anti_aliasing=False)
                    clippedROIt2w = hr_subroi[zone]>0
                    ssub_im[~clippedROIt2w] = np.nan

                    hr_w_tstand = normalize(hr_img)
                    w_temp_hr_sub = tanh(standardize(ssub_im))    
                    new_w_temp = tempIm_hr[nx1:nx2,ny1:ny2]*(1-weighting) + normalize(tempIm_hr[nx1:nx2,ny1:ny2]*w_temp_hr_sub)*weighting
                    new_w_temp = normalize(new_w_temp, [min(hr_w_tstand[hr_roi[zone]]),max(hr_w_tstand[hr_roi[zone]])])
                    hr_overlay_sub = hr_w_tstand[nx1:nx2,ny1:ny2]
                    hr_overlay_sub[clippedROIt2w] = new_w_temp[clippedROIt2w]
                    hr_w_tstand[nx1:nx2,ny1:ny2] = hr_overlay_sub
                    if zone == 'pz':
                        hr_full_savePZ[param] = copy.deepcopy(hr_w_tstand) # making a copy to save in pz and then combine with tz later for the "full"
                    
                    if figShow:
                        plt.figure()
                        plt.rcParams['figure.figsize'] = (10,10) # set default size of plots
                        plt.imshow(hr_w_tstand)
                    else:
                        plt.close()
                    if zonei == 3:
                        imageio.imwrite(zonePath +'/Overlay_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_full_slice_'+ str(hr_slice) + '_w' +str(weighting) + '_' + param +'.png', (hr_w_tstand*255).astype(np.uint8))
                    else:
                        imageio.imwrite(zonePath +'/Overlay_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_'+zone+'_slice_'+ str(hr_slice) + '_w' +str(weighting) + '_' + param +'.png', (hr_w_tstand*255).astype(np.uint8))
                    #^------------------------------------^  

                    # analysis
                    # Analysis of weighted - Histograms ------------------------------------,
                    # making legend & colours
                    colours = np.array(sns.color_palette(palette=cname, n_colors=8))
                    patchList = [] #create patches
                    patchList.append(mpatches.Patch(color='dodgerblue', label='Benign'))
                    for m in maligCompile[param]:
                        if np.sum(maligCompile[param][m][~np.isnan(maligCompile[param][m])]).astype(bool):
                            data_key = mpatches.Patch(color=legend_dict[gs_labels[m+2]], label=gs_labels[m+2]) 
                            patchList.append(data_key)
                    if zonei < 3:
                        row2print['Param'] = param
                        # --- Tanh on Standardized
                        row2print = showSepHist(w_tstand,param,roiB[zone],roiMdict[zone],ax[1,2], ax[2,2],patchList, colours,row2print,gs_labels)
                        row2print['Weighting fn'] = 'Tanh'
                        sheet = excel_output(sheet,toSave,row2print)

                        # --- sigmoid on Standardized
                        row2print = showSepHist(w_sstand,param,roiB[zone],roiMdict[zone],ax[1,3], ax[2,3],patchList, colours,row2print,gs_labels)
                        row2print['Weighting fn'] = 'Sigmoid'
                        sheet = excel_output(sheet,toSave,row2print)
                        
                        ax[1,0].imshow(baseim)
                        ax[1,0].set_title('Gleason Grading & Zones')
                        ax[1,0].axis('off')
                        ax[1,0].legend(handles=patchList, fontsize='small');
                        f.delaxes(ax[2,0])
                    if save:
                        imageio.imwrite(zonePath +'/LR_P'+str(pnum)+'_S' +str(snum)+'_T2Wroi_zone_'+zone+'_w' +str(weighting) + '_' + param +'.png', (w_tstand*255).astype(np.uint8))
                        # plt.savefig(zonePath +'/'+'P'+str(pnum) + '_S' +str(snum) + '_'+param+'_Weighted_' + zone + '.png')
                    if figShow:
                        plt.show()
                    else:
                        plt.close()
                
                if zonei == 3:
                    save = beforesave # reverting what the save flag was before!

    except NameError:
        print('Skipping P' + str(pnum) + ' S' +str(snum))
        shutil.rmtree(foldersavep)
    except:
        print('Error using min/max with current roi zone: ' + zone + ', number of values: ' + str(np.sum(img_decay[roi_zone])))
        raise
    return book, parameters


#### WIP Fn's

In [ ]:
import os
import re
import xlwt
from sklearn.metrics import roc_curve, auc
import shutil
from numpy import trapz
plt.figure().clear()
plt.close()

patNumList = [70] # np.arange(70,71)##np.arange(65,110+1) # 
snum = '7'
echo = 4
weight = [0.75]
weights = [0.75] #np.linspace(0,1,5)
saveFiles = True
figShow = True
cmap = 'Set1' #cubehelix
lwiPath = folder_nnls_decaes #_matlab # or decaes
norm = None

foldersavep = "C:/Users/candi/Documents/Research/10 Thesis/Figures/analysis/" #need "/" after
outputPath = makeDir(foldersavep+'/'+'Aug14')
saveExcel = 'analysisLV4.xls'
toSave = ['Patient','Slice','Echo', 'Zone','Norm?','Weighting %','Param','Weighting fn',\
          'Hist OVL (3+3)','Hist OVL (3+4)','Hist OVL (4+3)','Hist OVL (4+4)','Hist OVL (4+5)','Hist OVL (5+4)', \
          'KDE (3+3)','KDE (3+4)','KDE (4+3)','KDE (4+4)','KDE (4+5)','KDE (5+4)', \
          'AUC (3+3)','AUC (3+4)','AUC (4+3)','AUC (4+4)','AUC (4+5)','AUC (5+4)']

# initializing dictionary to save to excel
book,sheet = init_excel(toSave)
row2print = dict(zip(toSave, [None]*len(toSave)))

try:
    for p,pnum in enumerate(patNumList):
        pnum=int(pnum)
        patdirs = glob.glob(folder_roi+'/P*' + str(pnum) + '*') # getting the files that contain the patient number for the ROIs
        slices = [] # array that carries the slices available for the patient
        if patdirs:
            pnumPath = makeDir(outputPath+'/'+'P' + str(pnum))

        # # looping through the patient's slices
        # snums = []
        # for sliceFile in patdirs: # to keep things in order
        #     snums.append(re.compile("(?<=_S)\d*(?!>_)").findall(sliceFile)[0])
        #     snum_int = [int(x) for x in snums]
        #     snum_int.sort()
        #     snums = [str(x) for x in snum_int]
        # for snum in snums:
            snumPath = makeDir(pnumPath + '/S' + snum)
            book,parameters = runAnalysisWeightingGSmap(saveFiles,snumPath, sheet, row2print, figShow, T2Decay_data_folder, echo, lwiPath, folder_roi, folder_pz, folder_gs, HR_T2W_data_folder, pnum, int(snum), norm, weights, cmap)
            # printParamsAllROIs(T2Decay_data_folder, echo, lwiPath, folder_roi, folder_pz, folder_gs, pnum, int(snum), parameters, cname=None, save=False, analysisType='both')
                

    # Saving book to excel only if there are files
    if os.listdir(outputPath):
        book.save(outputPath + '/' + saveExcel)
        print('Analysis Finished. Location: ', outputPath)
    else:
        os.rmdir(outputPath)
        print('No files found to analyze.')
        shutil.rmtree(outputPath) # delete folder if error arises
except:
    book.save(outputPath + '/' + saveExcel)
    # shutil.rmtree(outputPath) # delete entire analysis folder if error arises
    raise